<a href="https://colab.research.google.com/github/lilyrausser/DS/blob/main/patient_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading Mx Data + filtering

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets

from sklearn.model_selection import train_test_split

from sklearn import metrics

In [ ]:
import pandas as pd

Data Processing

In [ ]:
new_data_total = pd.read_csv('patient_dropout.csv')
new_data_total

,diagnosis_1_codeset,diagnosis_code_1,patient_birth_year,patient_gender,patient_state,patient_id,date_of_service,days_supply,ndc_proprietaryname,npi,number_of_refills_authorized,patient_birth_year.1,patient_gender.1,patient_state.1,patient_pay_amount,plan_name,patient_id.1,diagnosis_code
0,ICD10,C73,1959,M,OH,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,6/29/2022,30,LENVIMA,1427068238,2,1959.0,M,OH,NaN,MEDCO SECONDARY PAYER NON-MED PART D,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73
1,ICD10,C73,1959,M,OH,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,6/29/2022,30,LENVIMA,1427068238,2,1959.0,M,OH,NaN,MEDCO SECONDARY PAYER NON-MED PART D,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73
2,ICD10,C73,1959,M,OH,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,6/29/2022,30,LENVIMA,1427068238,2,1959.0,M,OH,NaN,MEDCO SECONDARY PAYER NON-MED PART D,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73
3,ICD10,C73,1959,M,OH,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,6/29/2022,30,LENVIMA,1427068238,2,1959.0,M,OH,NaN,MEDCO SECONDARY PAYER NON-MED PART D,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73
4,ICD10,C73,1959,M,OH,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,6/29/2022,30,LENVIMA,1427068238,2,1959.0,M,OH,NaN,MEDCO SECONDARY PAYER NON-MED PART D,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237,ICD10,C73,1964,F,FL,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,11/18/2015,30,LENVIMA,1487640314,0,1964.0,U,FL,0.0,PCS ADVANCE PARADIGM,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,NaN
5238,ICD10,C73,1964,F,FL,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,11/18/2015,30,LENVIMA,1487640314,0,1964.0,U,FL,0.0,PCS ADVANCE PARADIGM,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,NaN
5239,ICD10,C73,1964,F,FL,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,11/18/2015,30,LENVIMA,1487640314,0,1964.0,U,FL,0.0,PCS ADVANCE PARADIGM,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,NaN
5240,ICD10,C73,1964,F,FL,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,11/18/2015,30,LENVIMA,1487640314,0,1964.0,U,FL,0.0,PCS ADVANCE PARADIGM,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,NaN


**Preprocessing:**

In [ ]:
new_data_total = pd.read_csv('best.csv')
new_data = new_data_total.drop(['diagnosis_code', 'patient_birth_year.1', 'patient_gender.1', 'patient_state.1', 'patient_id.1'], axis = 1)
new_data = new_data[['date_of_service', 'patient_id', 'diagnosis_code_1', 'ndc_proprietaryname', 'days_supply', 'number_of_refills_authorized', 'patient_birth_year', 'patient_gender', 'patient_state', 'plan_name', 'patient_pay_amount', 'diagnosis_1_codeset']]
new_data = new_data[new_data['ndc_proprietaryname'] == 'LENVIMA']
gender_map = {'U' : 0, 'F' : 1, 'M' : 2}
new_data['patient_gender'] = new_data['patient_gender'].map(gender_map)


new_data['patient_birth_year'] = new_data['patient_birth_year'].fillna(0).astype(int)
new_data['patient_pay_amount'] = new_data['patient_pay_amount'].fillna(0).astype(int)
new_data['patient_state'] = new_data['patient_state'].fillna(0)
new_data['plan_name'] = new_data['plan_name'].fillna(0)
new_data['date_of_service'] = pd.to_datetime(new_data['date_of_service'])

new_data['patient_id'] = new_data['patient_id'].fillna('').astype(str)
new_data['diagnosis_code_1'] = new_data['diagnosis_code_1'].fillna('').astype(str)
new_data['ndc_proprietaryname'] = new_data['ndc_proprietaryname'].fillna('').astype(str)
new_data['patient_state'] = new_data['patient_state'].fillna('').astype(str)
new_data['plan_name'] = new_data['plan_name'].fillna('').astype(str)
new_data['diagnosis_1_codeset'] = new_data['diagnosis_1_codeset'].fillna('').astype(str)

# find the number of unique patients in the table to iterate through
patient_len = new_data['patient_id'].nunique()

#finds the time stamp of the current day (when the code is run)
current_date = pd.Timestamp.today()

#extracts the date from that timestamp
today = current_date.date()

#initiates binary column at 0, assuming the patient is no longer using the medicine
new_data['currently_using_medicine'] = 0

#iterates through each patient ID
for patient in new_data['patient_id'].unique():
  #sums the total number of days a patient has been on a specific medicine
  total_days = new_data[new_data['patient_id'] == patient]['days_supply'].sum()

  #finds the most recent unique date that a patient has recieved service
  max_date_of_service = new_data[new_data['patient_id'] == patient]['date_of_service'].max()

  #if the max date of the patent is less than the current day and the date in addition to the summed days (total_days) that the medicine was used exceeds the current day:
  if (max_date_of_service.date() < today) and (max_date_of_service + pd.Timedelta(days=total_days) >= pd.Timestamp(today)):
    #declare that as a 1, as the patient is still using the medicine
    new_data.loc[new_data['patient_id'] == patient, 'currently_using_medicine'] = 1


#new_data['patient_birth_year'] = new_data['patient_birth_year'].astype(int)
total = today.year - new_data['patient_birth_year']

new = []
for i in new_data['patient_birth_year']:
  if (today.year - i == today.year):
    age = 0
    new.append(age)
  else:
    age = today.year - i
    new.append(age)

new_data['age'] = new


one_hot_plan = pd.get_dummies(new_data['plan_name'])
# Drop column B as it is now encoded
new_data = new_data.drop('plan_name',axis = 1)
# Join the encoded df
new_data = new_data.join(one_hot_plan)

new_data




,date_of_service,patient_id,diagnosis_code_1,ndc_proprietaryname,days_supply,number_of_refills_authorized,patient_birth_year,patient_gender,patient_state,patient_pay_amount,...,MEDICARE PART D MAGELLAN RX BASIC PDP,MEDICARE PART D NAVITUS HEALTH SOLUTIONS MA-PDP,MEDICARE PART D NRECA DRUG PROGRAM,MEDICARE PART D PRESCRIPTION SOLUTIONS,MEDICARE PART D RX BENEFIT DIRECT,PCS ADVANCE PARADIGM,PHARMACY DATA MANAGEMENT INC,PRESCRIPTION SOLUTIONS SUPER CARRIER PLAN,RX OPTIONS,UNITED HEALTHCARE COMMERCIAL
0,2022-06-29,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73,LENVIMA,30,2,1959,2,OH,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-06-29,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73,LENVIMA,30,2,1959,2,OH,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-06-29,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73,LENVIMA,30,2,1959,2,OH,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-06-29,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73,LENVIMA,30,2,1959,2,OH,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-06-29,112b32f9a70a99db25b9aae7119ac1b5c0ce31231d64d3...,C73,LENVIMA,30,2,1959,2,OH,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237,2015-11-18,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,C73,LENVIMA,30,0,1964,1,FL,0,...,0,0,0,0,0,1,0,0,0,0
5238,2015-11-18,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,C73,LENVIMA,30,0,1964,1,FL,0,...,0,0,0,0,0,1,0,0,0,0
5239,2015-11-18,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,C73,LENVIMA,30,0,1964,1,FL,0,...,0,0,0,0,0,1,0,0,0,0
5240,2015-11-18,3c29a91e0525b23a0c98457005b2a38c775747f1ae5eff...,C73,LENVIMA,30,0,1964,1,FL,0,...,0,0,0,0,0,1,0,0,0,0


**Classifier Pre-Processing:**

In [ ]:
new_data.columns

Index(['date_of_service', 'patient_id', 'diagnosis_code_1',
       'ndc_proprietaryname', 'days_supply', 'number_of_refills_authorized',
       'patient_birth_year', 'patient_gender', 'patient_state',
       'patient_pay_amount', 'diagnosis_1_codeset', 'currently_using_medicine',
       'age', '0', 'BLUE CROSS BLUE SHIELD FLORIDA COMMERCIAL PLAN',
       'CAREMARK', 'CASH PLAN CPR PLUS MANUFACTURER PROGRAMS',
       'CHRONIC DISEASE FUND', 'CIGNA PHARMACY SERVICES',
       'FEDERAL EMPLOYEE PROGRAM',
       'HORIZON BCBSNJ COMMERCIAL SELF FUNDED AND FULLY INSURED MEMBERS AND MEDIGAP',
       'HUMANA', 'INFORMEDRX NATIONAL', 'MANAGED CARE MEDICAID CALOPTIMA',
       'MEDCO', 'MEDCO SECONDARY PAYER NON-MED PART D',
       'MEDICARE PART D ALIGNMENT HEALTH PLAN',
       'MEDICARE PART D CIGNA PDP MAPD AND HEALTHSPRING PDP',
       'MEDICARE PART D ENVISION RX OPTIONS',
       'MEDICARE PART D EXPRESS SCRIPTS',
       'MEDICARE PART D HUMANA (PDP_MAPD_CAREPLUS)',
       'MEDICARE PART D LE

In [ ]:
from sklearn.preprocessing import LabelEncoder
#droping rows with missing values
new_data.dropna(inplace=True)

Y = new_data['currently_using_medicine']
#convert datetime column to numerical representation
categorical_cols = ['patient_id', 'diagnosis_code_1',
       'ndc_proprietaryname', 'days_supply', 'number_of_refills_authorized',
       'patient_birth_year', 'patient_gender', 'patient_state',
       'patient_pay_amount', 'diagnosis_1_codeset',
       'age', '0', 'BLUE CROSS BLUE SHIELD FLORIDA COMMERCIAL PLAN',
       'CAREMARK', 'CASH PLAN CPR PLUS MANUFACTURER PROGRAMS',
       'CHRONIC DISEASE FUND', 'CIGNA PHARMACY SERVICES',
       'FEDERAL EMPLOYEE PROGRAM',
       'HORIZON BCBSNJ COMMERCIAL SELF FUNDED AND FULLY INSURED MEMBERS AND MEDIGAP',
       'HUMANA', 'INFORMEDRX NATIONAL', 'MANAGED CARE MEDICAID CALOPTIMA',
       'MEDCO', 'MEDCO SECONDARY PAYER NON-MED PART D',
       'MEDICARE PART D ALIGNMENT HEALTH PLAN',
       'MEDICARE PART D CIGNA PDP MAPD AND HEALTHSPRING PDP',
       'MEDICARE PART D ENVISION RX OPTIONS',
       'MEDICARE PART D EXPRESS SCRIPTS',
       'MEDICARE PART D HUMANA (PDP_MAPD_CAREPLUS)',
       'MEDICARE PART D LEGACY MEDCO MEDICARE',
       'MEDICARE PART D MAGELLAN RX BASIC PDP',
       'MEDICARE PART D NAVITUS HEALTH SOLUTIONS MA-PDP',
       'MEDICARE PART D NRECA DRUG PROGRAM',
       'MEDICARE PART D PRESCRIPTION SOLUTIONS',
       'MEDICARE PART D RX BENEFIT DIRECT', 'PCS ADVANCE PARADIGM',
       'PHARMACY DATA MANAGEMENT  INC',
       'PRESCRIPTION SOLUTIONS SUPER CARRIER PLAN', 'RX OPTIONS',
       'UNITED HEALTHCARE COMMERCIAL']

label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

new_data_encoded = pd.get_dummies(new_data, columns=categorical_cols)
new_data_encoded['date_of_service'] = new_data_encoded['date_of_service'].astype('int64')


#dropping the dependent variable in order to define the rest of the input features
X = new_data_encoded.drop(['currently_using_medicine'], axis=1)

#dependent variable
Y = new_data_encoded['currently_using_medicine']

**Decision Tree Regressor Model:**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
#when would a patient fall off the drug

tree = DecisionTreeRegressor(criterion='squared_error', max_depth = 2)
model_1 = tree.fit(X_train, y_train)
y_pred_1 = model_1.predict(X_test)
mse = mean_squared_error(y_test, y_pred_1)
r_score_1 = r2_score(y_test, y_pred_1)
print("DecisionTreeRegressor Error:", mse)
print("DecisionTreeRegressor Accuracy:", r_score_1)

DecisionTreeRegressor Error: 0.0662114413870961
DecisionTreeRegressor Accuracy: 0.40134582796884366


**Decision Tree Classifier Model:**

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

tree_2 = tree.DecisionTreeClassifier()
model_2 = tree_2.fit(X_train, y_train)
y_pred_2 = model_2.predict(X_test)
mse_2 = mean_squared_error(y_test, y_pred_2)
r_score_2 = r2_score(y_test, y_pred_2)
print("DecisionTreeClassifier Error:", mse_2)
print("DecisionTreeClassifier Accuracy:", r_score_2)

DecisionTreeClassifier Error: 0.0017467248908296944
DecisionTreeClassifier Accuracy: 0.9842068965517241


**AdaBoostClassifier Model:**
*Random Forests*

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

#droping rows with missing values
new_data.dropna(inplace=True)
# Convert datetime column to numerical representation

#splits the data into training/testing
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

#creates model with 50 estimators and a learning rate of 1
#estimators: (trees with shallow depth) represents the number of tree stumps included in model
#learning_rate: controls the contribution of each stump to the model
AdaBoostClf = AdaBoostClassifier(n_estimators=50, learning_rate=1) #each stump contributes to the ensemble model with full weight

#fits the model to the training data
model = AdaBoostClf.fit(X_train, y_train)

#predict the dependent variable for the testing set
y_pred = model.predict(X_test)

#compares predicted values with actuals
mse_ada = mean_squared_error(y_test, y_pred)
print("AdaBoostClassifier Error:", mse_ada)
print("AdaBoostClassifier Accuracy:", metrics.accuracy_score(y_test, y_pred))

#differencebetween this and xgboost
#catboost/adaboost

AdaBoostClassifier Error: 0.0008733624454148472
AdaBoostClassifier Accuracy: 0.9991266375545852


**Confusion Matrix:**
*TP, FP, TN, FN*

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
cm = confusion_matrix(y_test, y_pred)

accuracy = (cm.ravel()[0] + cm.ravel()[3]) / sum(cm.ravel())
TPR = cm.ravel()[3] / (cm.ravel()[3] + cm.ravel()[2])
FPR = cm.ravel()[1] / (cm.ravel()[1] + cm.ravel()[0])
print(accuracy)
print(TPR)
print(FPR)

#TP FN
#FP TN

#precision: proportion of correctly predicted positive instances out of all instances predicted as positive
#recall: measures the proportion of correctly predicted positive instances out of all actual positives
#AUC-ROC


0.9991266375545852
1.0
0.007352941176470588
